In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#The stars
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Es momento de un juego de arquitecturas. Vamos a trabajar con un dataset de Kaggle, el [FIFA19 dataset](https://www.kaggle.com/karangadiya/fifa19). De los notebooks tambien saque algunos codigos copados.

In [ ]:
import pandas as pd
!wget http://raw.githubusercontent.com/IAI-UNSAM/ML_UNSAM/master/datasets/data_FIFA19.csv
data_FIFA=pd.read_csv('data_FIFA19.csv')
data_FIFA.info()

El objetivo va a ser obtener el Player Overall Ranking a partir de las distintas caracteristicas de los jugadores.

Hagamos un preprocesado rapido. Basicamente colapso las posiciones a una columna de 0,1,2 y tiro un monton de otras cosas.

In [ ]:
df = data_FIFA.copy()
drop_cols = df.columns[28:54]
df = df.drop(drop_cols, axis = 1)
df = df.drop(['Unnamed: 0','ID','Photo','Flag','Club Logo','Jersey Number','Joined','Special','Loaned From','Body Type', 'Release Clause',
               'Weight','Height','Contract Valid Until','Wage','Value','Club'], axis = 1)
df = df.dropna()
df.head()

In [ ]:
def right_footed(df):
    if (df['Preferred Foot'] == 'Right'):
        return 1
    else:
        return 0

#Create a simplified position varaible to account for all player positions
def simple_position(df):
    if (df['Position'] == 'GK'):
        return 0
    elif ((df['Position'] == 'RB') | (df['Position'] == 'LB') | (df['Position'] == 'CB') | (df['Position'] == 'LCB') | (df['Position'] == 'RCB') | (df['Position'] == 'RWB') | (df['Position'] == 'LWB') ):
        return 1
    elif ((df['Position'] == 'LDM') | (df['Position'] == 'CDM') | (df['Position'] == 'RDM')):
        return 2
    elif ((df['Position'] == 'LM') | (df['Position'] == 'LCM') | (df['Position'] == 'CM') | (df['Position'] == 'RCM') | (df['Position'] == 'RM')):
        return 2
    elif ((df['Position'] == 'LAM') | (df['Position'] == 'CAM') | (df['Position'] == 'RAM') | (df['Position'] == 'LW') | (df['Position'] == 'RW')):
        return 2
    elif ((df['Position'] == 'RS') | (df['Position'] == 'ST') | (df['Position'] == 'LS') | (df['Position'] == 'CF') | (df['Position'] == 'LF') | (df['Position'] == 'RF')):
        return 3
    else:
        return df.Position

df['Right_Foot'] = df.apply(right_footed, axis=1)
df['Simple_Position'] = df.apply(simple_position,axis = 1)

#Split the Work Rate Column in two
tempwork = df["Work Rate"].str.split("/ ", n = 1, expand = True) 
#Create new column for first work rate
df["WorkRate1"]= tempwork[0]   
#Create new column for second work rate
df["WorkRate2"]= tempwork[1]
#Drop original columns used
df = df.drop(['Work Rate','Preferred Foot','Real Face', 'Position','Nationality'], axis = 1)
df.head()

Vamos a separar en un par de subgrupos. El target es el Overall.

In [ ]:
y=np.asarray(df['Overall'])
name=np.asarray(df['Name'])
print(y.shape,name.shape)

Y primero vamos a utilizar unicamente los skills del jugador que son verdaderamente numericos

In [ ]:
num_cols=df.columns[4:-4]
df_num=df[num_cols]
df_num.head()

In [ ]:
df_num=np.asarray(df_num)
print(df_num.shape)

Y ahora quedemonos con el Simple Position para despues

In [ ]:
df_pos=np.asarray(df['Simple_Position']).reshape(-1,1)
df_pos.shape

Y dividamos todo bien:

In [ ]:
X_num_train_full, X_num_test,X_pos_train_full, X_pos_test, y_train_full, y_test = train_test_split(df_num,df_pos, y, random_state=42)
X_num_train, X_num_valid, X_pos_train, X_pos_valid, y_train, y_valid = train_test_split(X_num_train_full, X_pos_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_num_train = scaler.fit_transform(X_num_train)
X_num_valid = scaler.transform(X_num_valid)
X_num_test = scaler.transform(X_num_test)

Antes que nada, hagamos una regresion lineal para usar de baseline

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
lr = LinearRegression()
scores = cross_val_score(lr, X_num_train_full,y_train_full,scoring='neg_mean_squared_error')
print(-scores.mean(),scores.std())
lr.fit(X_num_train,y_train)
y_predict=lr.predict(X_num_valid)
plt.scatter(y_predict,y_valid)
plt.plot(y_valid,y_valid,color='black')
print(mean_squared_error(y_valid,y_predict))

Utilicemos una Red Neuronal con las siguientes caracteristicas:

- Una unica capa oculta de 10 neuronas, con activacion 'relu' e iniciacion 'he_normal'
- Una capa de salida con activacion 'relu' e iniciacion 'he_normal'

Entrene con error 'mse', SGD con learning rate 0.001,  batch_size=100, 200 epocas y un early_stopping de paciencia 10.

In [ ]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
model.summary()

In [ ]:
keras.utils.plot_model(model,show_shapes=True)

In [ ]:

plt.scatter(y_predict,y_valid)
plt.plot(y_valid,y_valid,color='black')
print(mean_squared_error(y_valid,y_predict))

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
#plt.gca().set_ylim(0, 10)
#plt.gca().set_xlim(5, 200)
plt.show()

Podemos bajar un poco el numero de parametros con un pequenio truco, las skip connections. La arquitectura es la siguiente

- Una capa oculta de 2 neuronas, con activacion 'relu' e iniciacion 'he_normal' que ve al input
- Una capa de salida con activacion 'relu' e iniciacion 'he_normal'

Entrene con error 'mse', Adam con learning rate de 0.01, batch_size=100, 250 epocas y un early_stopping de paciencia 10.

In [ ]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
model.summary()

In [ ]:
keras.utils.plot_model(model,show_shapes=True)

In [ ]:
plt.scatter(y_predict,y_valid)
plt.plot(y_valid,y_valid,color='black')
print(mean_squared_error(y_valid,y_predict))

Ahora, podemos mejorar esto de una manera sutil. Incorporemos un segundo output, la posicion. La idea es que buscando tanto interpolar el Overall y clasificar la posicion, la red neuronal sea todavia mejor.

Altere la arquitectura previa pero ahora utilice dos capas de outputs, agregando a la ya existente una con softmax e inicializacion Glorot uniforme.

Al compilar, hay que cambiar loss='mean_squared_error' por loss=['mean_squared_error','sparse_categorical_cross_entropy'] y agregar un loss_weight=[0.5,0.5]. El loss_weight marca el peso relativo entre los problemas. Si ponemos [0.9,0.1], el problema de clasificacion juega el rol de regularizacion. No nos interesa demasiado eso ya que no sobreajustamos tanto.

Al entrenar, hay que alimentar con dos outputs [y_train,X_pos_train], y lo mismo en validacion.

In [ ]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)


In [ ]:
model.summary()

In [ ]:
keras.utils.plot_model(model,show_shapes=True)

In [ ]:
plt.scatter(y_predict,y_valid)
plt.plot(y_valid,y_valid,color='black')
print(mean_squared_error(y_valid,y_predict))

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(X_pos_valid,np.argmax(pos_predict,axis=1))
cm